In [11]:
from data_tools import DBClient, FluxQuery
import pathlib
import os
import pickle
import pandas as pd
from datetime import datetime, timezone

client = DBClient()

Creating client with API Token: Hgnv70HqoniXv8D5S9rS-4jLIfhG4QlneuR4igLx31MOAB7aPo6NmzLAejbhDtDnBdSU2GQXmoNgfKpoJcqjnA==
Creating client with Org: 8a0b66d77a331e96


In [9]:
things_of_interest = ["VehicleVelocity", "AcceleratorPosition", "BatteryCurrent", "BatteryVoltage", "BatteryCurrentDirection", "MechBrakePressed"]

path = "/Users/joshuariefman/Solar/data_analysis/data_acquisition"
if not os.path.isdir("/Users/joshuariefman/Solar/data_analysis/data_acquisition"):
    path = input("Please enter path:")

folder = pathlib.Path(path)
del path

try:
    os.mkdir(folder / "csv")
except FileExistsError:
    pass

try:
    os.mkdir(folder / "pd")
except FileExistsError:
    pass

try:
    os.mkdir(folder / "ts")
except FileExistsError:
    pass

start_time = datetime(2024, 7, 15, 8, 0, 0, tzinfo=timezone.utc)
end_time = datetime(2024, 7, 21, 00, 0, 0, tzinfo=timezone.utc)

In [14]:
for thing_of_interest in things_of_interest:
    data_df = client.query_dataframe(FluxQuery().range(start_time.isoformat(), end_time.isoformat()).filter(field=thing_of_interest).from_bucket("CAN_log"))
    data_df['_time'] = pd.to_datetime(data_df['_time'])
    data_df.set_index('_time', inplace=True)

    data_df[thing_of_interest].to_csv(folder / "csv" / f"{thing_of_interest}.csv")
    data_df[thing_of_interest].to_pickle(folder / "pd" / f"{thing_of_interest}.pkl")

    data_ts = client.query_time_series(start_time, end_time, field=thing_of_interest)
    with open(folder / "ts" / f"{thing_of_interest}.pkl", "wb") as f:
        pickle.dump(data_ts, f, protocol=pickle.HIGHEST_PROTOCOL)